In [1]:
# -------------------------
# 📘 So sánh Haar Cascade vs DeepFace
# Người thực hiện: Ngô Quang Vũ - 197CT10170
# -------------------------

import cv2
import os
import datetime
import platform
from deepface import DeepFace

# -------------------------
# Haar Cascade setup
# -------------------------
haar_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(haar_path)

def detect_faces_haar(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def draw_haar_boxes(frame, faces):
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, "Haar", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    return frame

# -------------------------
# DeepFace setup
# -------------------------
def is_gui_available():
    return platform.system() != "Linux" or os.environ.get("DISPLAY") is not None

def analyze_deepface(frame):
    try:
        results = DeepFace.analyze(frame, actions=["age", "gender", "emotion"], enforce_detection=False)
        for face in results:
            region = face.get("region", {})
            if not region: continue
            x, y, w, h = region.values()
            gender = face.get("gender", "?")
            age = face.get("age", "?")
            emotion = face.get("dominant_emotion", "?")
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            label = f"{gender}, {age}, {emotion}"
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    except Exception as e:
        print("⚠️ DeepFace error:", e)
    return frame

# -------------------------
# Xử lý chung cho từng frame
# -------------------------
def process_frame(frame):
    # Clone frame cho 2 phương pháp
    haar_frame = frame.copy()
    deepface_frame = frame.copy()

    # Haar Cascade
    faces = detect_faces_haar(haar_frame)
    haar_frame = draw_haar_boxes(haar_frame, faces)

    # DeepFace
    deepface_frame = analyze_deepface(deepface_frame)

    # Ghép 2 frame để so sánh
    combined = cv2.hconcat([haar_frame, deepface_frame])
    return combined

# -------------------------
# Xử lý ảnh
# -------------------------
def compare_on_image(image_path):
    if not os.path.exists(image_path):
        print(" Không tìm thấy ảnh:", image_path)
        return

    image = cv2.imread(image_path)
    combined = process_frame(image)

    if is_gui_available():
        cv2.imshow("So sánh Haar vs DeepFace", combined)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    os.makedirs("result_compare", exist_ok=True)
    out_path = os.path.join("result_compare", f"compare_{os.path.basename(image_path)}")
    cv2.imwrite(out_path, combined)
    print(f"💾 Đã lưu ảnh so sánh: {out_path}")

# -------------------------
# Xử lý video
# -------------------------
def compare_on_video(video_path):
    if not os.path.exists(video_path):
        print(" Không tìm thấy video:", video_path)
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(" Không thể mở video.")
        return

    os.makedirs("result_compare", exist_ok=True)
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        combined = process_frame(frame)

        if is_gui_available():
            cv2.imshow("So sánh Haar vs DeepFace", combined)

        out_path = f"result_compare/frame_{frame_id}.jpg"
        cv2.imwrite(out_path, combined)

        if is_gui_available() and cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if is_gui_available():
        cv2.destroyAllWindows()
    print("📽️ Đã kết thúc xử lý video.")

# -------------------------
# Xử lý webcam
# -------------------------
def compare_on_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print(" Không thể mở webcam.")
        return

    os.makedirs("result_compare", exist_ok=True)
    frame_id = 0

    print("📡 Đang quét webcam... Nhấn 'e' hoặc ESC để thoát.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        combined = process_frame(frame)

        out_path = f"result_compare/webcam_{frame_id}.jpg"
        cv2.imwrite(out_path, combined)

        if is_gui_available():
            cv2.imshow("So sánh Haar vs DeepFace", combined)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('e') or key == 27:
            print("👋 Đã thoát webcam.")
            break

    cap.release()
    if is_gui_available():
        cv2.destroyAllWindows()
    print(" Kết thúc quét webcam.")

# -------------------------
# Menu chính
# -------------------------
def main():
    while True:
        print("\n=== So sánh Haar Cascade và DeepFace ===")
        print("1. So sánh từ Ảnh")
        print("2. So sánh từ Video")
        print("3. So sánh từ Webcam")
        print("4. Thoát")
        choice = input("👉 Nhập lựa chọn (1/2/3/4): ")

        if choice == '1':
            path = input(" Nhập đường dẫn ảnh: ")
            compare_on_image(path)
        elif choice == '2':
            path = input(" Nhập đường dẫn video: ")
            compare_on_video(path)
        elif choice == '3':
            compare_on_webcam()
        elif choice == '4':
            print(" Thoát chương trình.")
            break
        else:
            print(" Lựa chọn không hợp lệ.")

if __name__ == "__main__":
    main()




=== So sánh Haar Cascade và DeepFace ===
1. So sánh từ Ảnh
2. So sánh từ Video
3. So sánh từ Webcam
4. Thoát
 Thoát chương trình.
